In [3]:
data = """O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""
data = open('puzzle.data').read()

def parse(data: str) -> tuple[set[complex], set[complex]]:
    rounds, cubes = set(), set()
    for y, line in enumerate(data.splitlines()):
        for x, c in enumerate(line):
            if c == '#':
                cubes.add(complex(x, y))
            elif c == 'O':
                rounds.add(complex(x, y))
    return tuple(rounds), tuple(cubes)

def tilt_north(rounds, cubes):
    new_set = set()
    for rock in sorted(rounds, key=lambda p: p.imag):
        y = 0
        for y in range(int(rock.imag), -1, -1):
            if complex(rock.real, y) in new_set or complex(rock.real, y) in cubes:
                new_set.add(complex(rock.real, y + 1))
                break
        else:
            new_set.add(complex(rock.real, 0))
    return new_set

def solve1(data: str):
    rounds, cubes = parse(data)
    rounds = tilt_north(rounds, cubes)
    rows = max(p.imag for p in rounds | set(cubes))
    load = [rows - p.imag + 1 for p in rounds]
    return int(sum(load))

solve1(data)

109654

In [4]:
def tilt_south(rounds, cubes):
    new_set = set()
    rows = max(p.imag for p in rounds | set(cubes))
    for rock in sorted(rounds, key=lambda p: p.imag, reverse=True):
        y = rows
        for y in range(int(rock.imag), int(rows + 1)):
            if complex(rock.real, y) in new_set or complex(rock.real, y) in cubes:
                new_set.add(complex(rock.real, y - 1))
                break
        else:
            new_set.add(complex(rock.real, rows))
    return new_set

def tilt_west(rounds, cubes):
    new_set = set()
    for rock in sorted(rounds, key=lambda p: p.real):
        x = 0
        for x in range(int(rock.real), -1, -1):
            if complex(x, rock.imag) in new_set or complex(x, rock.imag) in cubes:
                new_set.add(complex(x + 1, rock.imag))
                break
        else:
            new_set.add(complex(0, rock.imag))
    return new_set

def tilt_east(rounds, cubes):
    new_set = set()
    columns = max(p.real for p in rounds | set(cubes))
    for rock in sorted(rounds, key=lambda p: p.real, reverse=True):
        x = columns
        for x in range(int(rock.real), int(columns + 1)):
            if complex(x, rock.imag) in new_set or complex(x, rock.imag) in cubes:
                new_set.add(complex(x - 1, rock.imag))
                break
        else:
            new_set.add(complex(columns, rock.imag))
    return new_set

from functools import cache

def dump(rounds, cubes):
    rounds, cubes = set(rounds), set(cubes)
    for y in range(int(max(p.imag for p in rounds | cubes) + 1)):
        for x in range(int(max(p.real for p in rounds | cubes) + 1)):
            if complex(x, y) in cubes:
                print('#', end='')
            elif complex(x, y) in rounds:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print('\n')

@cache
def cycle(rounds, cubes):
    rounds = tilt_north(rounds, cubes)
    rounds = tilt_west(rounds, cubes)
    rounds = tilt_south(rounds, cubes)
    return tuple(tilt_east(rounds, cubes))

@cache
def batch(rounds, cubes, cycles: int):
    for _ in range(cycles):
        rounds = cycle(rounds, cubes)
    return tuple(rounds)

def solve2(data: str):
    rounds, cubes = parse(data)
    
    for _ in range(1_000_000):
        rounds = batch(rounds, cubes, 1000)
    rows = max(p.imag for p in rounds + cubes)
    load = [rows - p.imag + 1 for p in rounds]
    return int(sum(load))

solve2(data)

94876